<a href="https://colab.research.google.com/github/Akshatgarg221/Web-dev/blob/main/.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date as date_n
import matplotlib.pyplot as plt
import plotly.graph_objects as go
start='2018-01-01'
end='2022-01-01'
ticker='AAPL'
rrfticker="^TNX"
df=yf.download(ticker,start,end)
drf=yf.download(rrfticker,start,end)
df.reset_index(inplace=True)
import warnings
warnings.filterwarnings("ignore")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
#getting risk free return
drf['return']=0
for i in range(0,len(drf)):
  drf['return'][i]=(drf['Close'][i]-drf['Close'][i-1])/drf['Close'][i]
drf['return']=drf['return']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Closing Price', line=dict(color='blue')))

fig.update_layout(
    title='Closing Prices Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Closing Price'),
    legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
    template='plotly_white'
)
fig.show()

In [ ]:
def macd(df,shortp,longp,signal):
  df['emalong']=df['Close'].ewm(span=longp,adjust=False).mean()
  df['emashort']=df['Close'].ewm(span=shortp,adjust=False).mean()
  df['macd']=df['emashort']-df['emalong']
  df['signalline']=df['macd'].ewm(span=signal, adjust=False).mean()
  return df

df=macd(df,12,26,9)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,emalong,emashort,macd,signalline
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.615879,102223600,43.064999,43.064999,0.000000,0.000000
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.608818,118071600,43.064443,43.063845,-0.000598,-0.000120
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.797432,89738400,43.078744,43.093638,0.014894,0.002883
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.261932,94640000,43.128466,43.194617,0.066150,0.015537
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.108677,82271200,43.162469,43.255060,0.092592,0.030948


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[:200], y=df['macd'], mode='lines', name='MACD', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index[:200], y=df['signalline'].iloc[:200], mode='lines', name='Signal line', line=dict(color='red')))

fig.update_layout(
     title='INDICATOR',
     xaxis=dict(title='Date'),
     yaxis=dict(title='Portfolio Value'),
     legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
     template='plotly_white'
 )
fig.show()

In [ ]:
df['signal']=0
for i in range(0,len(df)):
  if(df['macd'][i]>df['signalline'][i] and df['macd'][i-1]<df['signalline'][i-1]):
    df['signal'][i]=1
  if(df['macd'][i]<df['signalline'][i] and df['macd'][i-1]>df['signalline'][i-1]):
    df['signal'][i]=-1
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,emalong,emashort,macd,signalline,signal
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,40.615879,102223600,43.064999,43.064999,0.000000,0.000000,0
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,40.608818,118071600,43.064443,43.063845,-0.000598,-0.000120,0
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,40.797432,89738400,43.078744,43.093638,0.014894,0.002883,1
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.261932,94640000,43.128466,43.194617,0.066150,0.015537,0
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.108677,82271200,43.162469,43.255060,0.092592,0.030948,0


In [ ]:
ls=[]
le=[]
ss=[]
se=[]
current=0
for i in range(0,len(df)):
  if(current==0):
    current=df['signal'][i]
    if(current==1):
      ls.append(i)
    elif(current==-1):
      ss.append(i)
  elif(current==1 and df['signal'][i]==1):
    df['signal'][i]=0
  elif(current==-1 and df['signal'][i]==-1):
    df['signal'][i]=0
  elif(current==1 and df['signal'][i]==-1):
    le.append(i)
    current=0
  elif(current==-1 and df['signal'][i]==1):
    se.append()
    current=0

df['signal'][len(df)-1]-=df['signal'].sum()
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,emalong,emashort,macd,signalline,signal
1003,2021-12-27,177.089996,180.419998,177.070007,180.330002,177.824493,74919600,168.900161,173.951370,5.051209,5.174498,0
1004,2021-12-28,180.160004,181.330002,178.529999,179.289993,176.798920,79144300,169.669778,174.772697,5.102919,5.160182,0
1005,2021-12-29,179.330002,180.630005,178.139999,179.380005,176.887680,62348900,170.389054,175.481514,5.092459,5.146637,0
1006,2021-12-30,179.470001,180.570007,178.089996,178.199997,175.724060,59773000,170.967643,175.899742,4.932099,5.103730,0
1007,2021-12-31,178.089996,179.229996,177.259995,177.570007,175.102829,64062300,171.456707,176.156706,4.699999,5.022983,0


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='NO Trade', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index[ls[0]:le[0]], y=df['Close'].iloc[ls[0]:le[0]], mode='lines', line=dict(color='green'),name='LONG TRADE'))
if(len(ss)):
  fig.add_trace(go.Scatter(x=df.index[ss[0]:se[0]], y=df['Close'].iloc[ss[0]:se[0]], mode='lines', line=dict(color='red'),name='SHORT TRADE'))
for i in range(1,len(le)):
  fig.add_trace(go.Scatter(x=df.index[ls[i]:le[i]], y=df['Close'].iloc[ls[i]:le[i]], mode='lines', line=dict(color='green'),showlegend=False))

for i in range(1,len(se)):
  fig.add_trace(go.Scatter(x=df.index[ss[i]:se[i]], y=df['Close'].iloc[ss[i]:se[i]], mode='lines', line=dict(color='red'),showlegend=False))

fig.update_layout(
    title='Closing Prices Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Closing Price'),
    legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
    template='plotly_white'
)
fig.show()

In [ ]:
enteryprice=[]
exitprice=[]
enterydate=[]
exitdate=[]
typetrade=[]
quantity=[]
returns=[]
drawdown=[]
dip=[]
riskfree=[]
df['Quantity']=0
df['Cash']=0
df['portfolio value']=0
amount=10000
position=0
noofshare=0
min=0
max=0
drawlong=0
drawshort=0
rrf=1
for i in range(0,len(df)):
  rrf*=1+drf['return'][i]
  sig=df['signal'][i]
  price=df['Close'][i]
  if(min>price):
    min=df['Low'][i]
  if(max<price):
    max=df['High'][i]
  if(((max-min)*100)/max>drawlong):
    drawlong=((max-min)*100)/max
  if(((min-max)*100)/min>drawshort):
    drawshort=((max-min)*100)/min


  if(sig==1):
    if(position==0):
      noofshare=amount//price
      amount=amount%price
      position=1

      #ENTERING TRADE DETAILS
      enteryprice.append(price)
      enterydate.append(df['Date'][i])
      typetrade.append(1)
      quantity.append(noofshare)
      #drawdown starting
      min=price
      max=price
      drawlong=0
      drawshort=0
      rrf=1

    if(position==-1):
      amount+=noofshare*(enteryprice[-1]-price)
      noofshare=0
      position=0

      #ENTERING TRADE DETAILS
      exitprice.append(price)
      exitdate.append(df['Date'][i])
      drawdown.append(drawshort)#drawdown update
      dip.append(((enteryprice[-1]-max)*100)/enteryprice[-1])
      riskfree.append((rrf-1)*100)



  if(sig==-1):
    if(position==0):
      noofshare=amount//price
      position=-1

      #ENTERING TRADE DETAILS
      enteryprice.append(price)
      enterydate.append(df['Date'][i])
      typetrade.append(-1)
      quantity.append(noofshare)
      #drawdonw start
      min=price
      max=price
      drawlong=0
      drawshort=0
      rrf=1


    if(position==1):
      amount+=noofshare*(price)
      noofshare=0
      position=0

      #ENTERING TRADE DETAILS
      exitprice.append(price)
      exitdate.append(df['Date'][i])
      returns.append((price-enteryprice[-1])/enteryprice[-1])
      drawdown.append(drawlong)#drawdown update
      dip.append(((enteryprice[-1]-min)*100)/enteryprice[-1])
      riskfree.append((rrf-1)*100)
  if(position==1):
    df['Quantity'][i]=noofshare
  if(position==-1):
    df['Quantity'][i]=-noofshare
  if(position==-1):
    df['Cash'][i]=amount-noofshare*enteryprice[-1]#Taking in account that for selling amount of share sold is kept as security
  else:
    df['Cash'][i]=amount
  if(position==1):
    df['portfolio value'][i]=amount+noofshare*price
  elif(position==-1):
    df['portfolio value'][i]=amount+noofshare*(enteryprice[-1]-price)
  else:
    df['portfolio value'][i]=amount




print("Amount at the end of trade= ₹",amount)
print("Return percentage of trade=",(amount-10000)/100,"%")


Amount at the end of trade= ₹ 33551.12907028198
Return percentage of trade= 235.51129070281982 %


In [ ]:
td=[]
for i in range(0,len(enterydate)):
  td.append((exitdate[i]-enterydate[i]).days)


In [ ]:
returnsp=np.array(returns)*100
dic={ 'Entry Date': enterydate,'Exit Date': exitdate,'Entery price':enteryprice,'Exit price':exitprice,'Trade type':typetrade,'Quantity':quantity,'Trade duration':td,
     'reutnrs %':returnsp,'Drawdown':drawdown,'Dip':dip,'Risk free rate':riskfree}
tradedetail=pd.DataFrame(dic)
tradedetail.head()
# print(len(exitprice))

,Entry Date,Exit Date,Entery price,Exit price,Trade type,Quantity,Trade duration,reutnrs %,Drawdown,Dip,Risk free rate
0,2018-01-04,2018-01-25,43.257500,42.777500,1,231.0,21,-1.109633,5.313718,1.444836,6.638540
1,2018-02-14,2018-03-19,41.842499,43.825001,1,236.0,33,4.738010,8.235103,0.000000,-2.590361
2,2018-04-10,2018-04-20,43.312500,41.430000,1,239.0,10,-4.346320,7.550022,4.513713,5.394995
3,2018-05-02,2018-05-25,44.142502,47.145000,1,224.0,23,6.801832,7.249035,0.000000,-1.327885
4,2018-06-06,2018-06-08,48.494999,47.924999,1,218.0,2,-1.175378,2.170322,2.170322,-1.297733


In [ ]:
df=df.drop(['Adj Close','emalong','emashort','macd','signalline'],axis=1)#just to shorten data
#plotting graph for daily noofshares and
df.head(10)

,Date,Open,High,Low,Close,Volume,signal,Quantity,Cash,portfolio value
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,102223600,0,0,10000.00000,10000.000000
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,118071600,0,0,10000.00000,10000.000000
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,89738400,1,231,7.51757,10000.000000
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,94640000,0,231,7.51757,10113.767570
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,82271200,0,231,7.51757,10076.230423
5,2018-01-09,43.637501,43.764999,43.352501,43.582500,86336000,0,231,7.51757,10075.075176
6,2018-01-10,43.290001,43.575001,43.250000,43.572498,95839600,0,231,7.51757,10072.764683
7,2018-01-11,43.647499,43.872501,43.622501,43.820000,74670800,0,231,7.51757,10129.937500
8,2018-01-12,44.044998,44.340000,43.912498,44.272499,101672400,0,231,7.51757,10234.464859
9,2018-01-16,44.474998,44.847500,44.035000,44.047501,118263600,0,231,7.51757,10182.490211


In [ ]:
def plot(df):
  #Plotting graph of Portfolio value
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df['Date'], y=df['portfolio value'], mode='lines', name='MACD', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=df.index[:200], y=df['Portfo'].iloc[:200], mode='lines', name='Signal line', line=dict(color='red')))

  fig.update_layout(
     title='Portfolio Value Over Time',
     xaxis=dict(title='Date'),
     yaxis=dict(title='Portfolio Value'),
     legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
     template='plotly_white'
 )
  fig.show()

 #ploting graph of no of shares owned
  fig1 = go.Figure()
  fig1.add_trace(go.Bar(
     y=df['Quantity'],
     x=df['Date'],
     marker=dict(
         color='rgba(0, 0, 245, 0.6)',
         line=dict(color='rgba(0, 0, 245, 1.0)', width=3)
     )
 ))
  fig1.update_layout(
     title='Share owned Over Time',
     xaxis=dict(title='Date'),
     yaxis=dict(title='No of shares'),
     legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
     template='plotly_white'
 )
  fig1.show()

In [ ]:
tradedetail['riskadjusted']=tradedetail['reutnrs %']-tradedetail['Risk free rate']
returnmeans=tradedetail['riskadjusted'].mean()
returnstd=tradedetail['riskadjusted'].std(ddof=1)
sharpe=returnmeans/returnstd*(252**0.5)
benchmarkreturn=(df['Close'][len(df)-1]-df['Close'][0])/df['Close'][0]
benchmarkreturn*=100
noofclosedtrade=len(tradedetail)
wintrade=0
downside=[]

for i in range(0,len(tradedetail)):
  if(tradedetail['reutnrs %'][i]>0):
      wintrade+=1
  if(tradedetail['riskadjusted'][i]<0):
    downside.append(tradedetail['riskadjusted'][i])
downside=np.array(downside)
returnstd=downside.std(ddof=1)
sortino=returnmeans/returnstd*(252**0.5)

loosetrade=noofclosedtrade-wintrade
maxdrawdown=tradedetail['Drawdown'].max()
avgdrawdown=tradedetail['Drawdown'].mean()
maxdip=tradedetail['Dip'].max()
avgdip=tradedetail['Dip'].mean()
maxtime=tradedetail['Trade duration'].max()
avgtime=tradedetail['Trade duration'].mean()
totaltime=tradedetail['Trade duration'].sum()
grossp=amount-10000
netp=grossp-20*noofclosedtrade
portfoliomax=df['portfolio value'].max()
portfoliomin=df['portfolio value'].min()

backnumbers=[(amount-10000)/100,benchmarkreturn,sharpe,sortino,10000,amount,grossp,netp,portfoliomax,portfoliomin,noofclosedtrade,wintrade,loosetrade,maxdrawdown,avgdrawdown,maxdip,avgdip]
backname=['RETURNS %','BENCHMARK RETURNS','SHARPE RATIO','SORTINO RATIO','AMOUNT INVESTED','FINAL AMOUNT','GROSS RETURNS',
             'NET RETURNS','PORTOFOLIO MAX VALUE',"PORTFLIO MINIMUM VALUE","NO OF TRADES",
             "NO OF WINTRADE","NO OF LOOSETRADE","MAXIMUM DRAWDOWN %","AVERAGE DRAWDOWN %"
             ,"MAXIMUM DIP %","AVERAGE DIP %"]
backtestingdict={"BACKTESTING BENCHMARKS":backname,"RESULTS":backnumbers}
backdf=pd.DataFrame(backtestingdict)
# print("KEY BACKTESTING RETURNS =>")
# print()
# print("RETURNS: ",(amount-10000)/100)
# print("BENCHMARK RETURNS: ",benchmarkreturn)
# print("SHARPE RATIO: ",sharpe)
# print("AMOUNT INVESTED: 10000")
# print("GROSS PROFIT: ",grossp)
# print("NET PROFIT",netp)
# print("PORTFLIO MAX VALUE: ",portfoliomax)
# print("PORTFLIO MINIMUM VALUE",portfoliomin)
# print("NO OF CLOSED TRADE: ",noofclosedtrade)
# print("NO OF WINTRADE: ",wintrade)
# print("NO OF LOOSETRADE: ",loosetrade)
# print("MAX DRAWDOWN: ",maxdrawdown)
# print("AVERAGE DRAWDOWN: ",avgdrawdown)
# print("MAX DIP: ",maxdip)
# print("AVERAGE DIP: ",avgdip)
# backdf.resetindex(inplace=True)
# print(portfoliomin)
# print(portfoliomax)

plot(df)
backdf



,BACKTESTING BENCHMARKS,RESULTS
0,RETURNS %,235.511291
1,BENCHMARK RETURNS,312.330229
2,SHARPE RATIO,7.055127
3,SORTINO RATIO,21.396466
4,AMOUNT INVESTED,10000.000000
5,FINAL AMOUNT,33551.129070
6,GROSS RETURNS,23551.129070
7,NET RETURNS,22831.129070
8,PORTOFOLIO MAX VALUE,35179.888592
9,PORTFLIO MINIMUM VALUE,8603.643436
